# Numpy version -- untested!

In this notebook we define the following functions:

|Function name| Description|
| -- | -- |
| `get_gmm_j` | Returns the $j^{th}$ Gaussian mixture model|
| `get_C_j` | Returns the $j^{th}$ GMM covariance matrix|
| `gmm_kernel` | Returns the sum of the $N_{GMM}$ Gaussian mixture models|
| `get_A_matrix` | Return the column-wise concatenated $A$ matrix|
| `compute_f_star` | Computes the $f^{*}$, the vector of stellar fluxes|
| `init_N_stars` | Initialize the locations of $N_stars$|
| `lnlike` | Compute the log likelihood|


These functions have the following inputs and outputs:

|Function name| Input| Output |
| -- | -- | -- |
| `get_gmm_j` | $x_j, y_j$, $C_j$ | $\mathcal{G_j}$|
| `get_C_j` | $\phi_{a,b,c,j}$ | $C_j$|
| `gmm_kernel` | $\mathcal{G_j}$, $z_j$| $\mathcal{K}$|
| `get_C_matrix` | $\sigma_r$, $N_{pix}$| $\mathsf{C}$|
| `get_A_matrix` | $\mathcal{K}$, $(x_c, y_c)$'s| $\mathsf{A}$|
| `get_f_star` | $\mathsf{A}$, $\mathsf{C}$, $\mathsf{d}$|$f^{*}$|
| `init_N_stars` | catalog *or* image| $(x_c,y_c)$| 
| `lnlike` | all parameters |log likelihood|

In [1]:
# %load ~/Desktop/defaults.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
def get_C_j(phi_a, phi_b, phi_c):
    '''Return the GMM covariance matrix given phi parameters'''
    v11 = np.exp(2.0*phi_a)
    v12 = phi_c*np.exp(phi_a)
    v22 = phi_c**2 + np.exp(2.0*phi_b)
    C_j = np.matrix([[v11, v12],[v12, v22]])
    return C_j

In [3]:
def get_gmm_j(x, y, x_j, y_j, C_j):
    '''Returns the j^th normalized Gaussian mixture model'''
    x = x - x_j
    y = y - y_j
    rr = np.stack([x,y], axis=0)
    exp_arg = np.sum(rr * np.linalg.solve(C_j, rr), axis=0)
    gmm_j_raw = np.exp(-0.5 * exp_arg)
    print(exp_arg.shape)
    normalization = 2.0*np.pi*np.sqrt(np.linalg.det(C_j))
    gmm_j = 1.0/normalization * gmm_j_raw
    return gmm_j

In [4]:
def gmm_z_js(theta_js):
    '''Return the GMM z_j's given components and theta_js'''
    z_js = np.ones(len(theta_js)+1)
    for j, theta_j in enumerate(theta_js):
        z_js[j+1] = z_js[j] + np.exp(-theta_js[j])
    z_js = z_js / np.sum(z_js)
    return z_js

In [5]:
def compute_model_a(x, y, z_js, x_js, y_js, phi_as, phi_bs, phi_cs):
    '''Takes in x, y and GMM kernel params, returns model'''
    N_GMM = x_js.shape
    gmm_out = 0.0
    for j in range(N_GMM):
        C_j = get_C_j(phi_as[j], phi_bs[j], phi_cs[j])
        gmm_out += z_js[j] * get_gmm_j(x, y, x_js[j], y_js[j], C_js[j])
    return gmm_out

In [6]:
def get_C_matrix(sigma_r, Npix):
    '''Return the noise covariance matrix'''
    return np.eye(Npix) * sigma_r**2

In [7]:
def get_f_star_nocov(A, sigma_r, d):
    '''Compute the profile likelihood of the stellar fluxes'''
    #For now ignore the covariance matrix, assume homoscedastic
    ATA = np.dot(A.T, A / sigma_r)
    f_star = np.linalg.solve(ATA, np.dot(A.T, d/sigma_r))
    return f_star

In [8]:
def get_A_matrix(x, y, x_c, y_c, kernel_params):
    '''Return the column-wise concatenated A matrix'''
    N_stars = len(x_c)
    xx = x[:, np.newaxis] - x_c.T
    yy = y[:, np.newaxis] - y_c.T
    A_matrix = np.zeros(xx.shape)
    for i in range(N_stars):
        A_matrix[:, i] = compute_model_a(xx[:,i], yy[:,i],*kernel_params)
    return A_matrix

In [9]:
def split_params(params, N_star, N_GMM):
    '''Split/clean all parameters into star and Kernel parameters'''
    star_params = params[0:N_star*2].reshape(N_star)
    kern_params = np.stack([[1, 0, 0], params[N_star*2:]]).reshape(N_GMM)
    kern_params[1:, 0] = gmm_z_js(kern_params[1:, 0])
    return star_params, kern_params

In [10]:
def init_N_stars(x_cg, y_cg, f_g):
    '''Initialize the starting locations of the guesses'''
    return (x_cg, y_cg, f_g)

In [11]:
# Data preparation.  This should happen outside the likelihood!

sigma_r = 4.0 # read noise
data_2D = np.ones((100,100))
nx_pix, ny_pix = data_2D.shape
xpix = np.arange(0, nx_pix, 1)
ypix = np.arange(0, ny_pix, 1)

xv, yv = np.meshgrid(xpix, ypix)
x = xv.reshape(-1)
y = yv.reshape(-1)

data = data_2D.reshape(-1)

N_pix = np.shape(data)

In [12]:
def lnlike(params):
    '''Return the likelihood given the parameters'''
    star_params, kern_params = split_params(params)
    
    x_c, y_c = star_params[0, :], star_params[1, :]
    
    # Just a homoscedastic noise matrix for now
    #C_noise_matrix = get_C_matrix(sigma_r, N_pix)
    
    # Get the design matrix for stellar fluxes
    A_matrix = get_A_matrix(x, y, x_c, y_c, kern_params)
    
    # Compute the profile likelihood for stellar fluxes
    f_star = get_f_star_nocov(A_matrix, sigma_r, data)
    
    model = np.dot(A_matrix, f_star)
    resid = data - model
    lnlike_out = np.dot(resid.T, resid / yerr**2)
    return 0.0

Warning! Untested version!